In [1]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

/home/prawater/.local/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage
import base64

image_file = "./badam.jpg"
with open(image_file, "rb") as image_file:
    image_data = image_file.read()
base64_image = base64.b64encode(image_data).decode('utf-8')

image_url = f"data:image/jpeg;base64,{base64_image}"

class CalorieAnalysis(BaseModel):
    name: str = Field(description="name of food sample in the image in kcal")
    calories: str = Field(description="calorie content of food sample in g")
    proteins: str = Field(description="protein content of food sample in g")
    fats: str = Field(description="fats content of food sample in g")
    carbohydrates: str = Field(description="carbohydrate content of food sample in g")
    dietary_fiber: str = Field(description="fiber content of food sample in g")
    sugars: str = Field(description="sugar content of food sample in g")
    sodium: str = Field(description="sodium content of food sample in mg")
    cholesterol: str = Field(description="cholesterol content of food sample in mg")

json_parser = JsonOutputParser(pydantic_object=CalorieAnalysis)

text_prompt = f"You are an expert dietician and you are helping your client estimate the amount of calories they are about to consume. The client requires accurate information for helping them achieve their fitness goals which you are responsible for. Estimate the amount of calories in this image. Step 1: Identify the food item in this image. Step 2: Estimate the weight (in grams) of the food item. Step 3: Now estimate the calorie count using the weight from step 2. Step 4: Now divide the content into calories, proteins, fats, etc. {json_parser.get_format_instructions()}"

combined_prompt = HumanMessage(content=[
        {"type": "image_url", "image_url": image_url},
        {
            "type": "text",
            "text": text_prompt
        }
    ])

response = model.invoke([combined_prompt])
parsed_response = json_parser.parse(response.content)

print(parsed_response)

{'name': 'Almonds', 'calories': '200', 'proteins': '6g', 'fats': '14g', 'carbohydrates': '6g', 'dietary_fiber': '4g', 'sugars': '1g', 'sodium': '0mg', 'cholesterol': '0mg'}
